In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\\labelled_bird_sample.csv")

print(df.shape)

df.head()

(103992, 15)


,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,seas_reg_rare
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common


In [3]:
import category_encoders as ce

In [4]:
train, test = train_test_split(df, test_size=0.2, stratify=df['seas_reg_rare'])

train.shape, test.shape

((83193, 15), (20799, 15))

In [5]:
features = ['name', 'season', 'RegionName', 'latitude', 'longitude']
target = 'seas_reg_rare'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((83193, 5), (83193,), (20799, 5), (20799,))

In [6]:
from sklearn.cluster import KMeans

In [12]:
encoder = ce.OrdinalEncoder()

cat_data = encoder.fit_transform(df[['name', 'season', 'RegionName']])
num_data = df[['latitude', 'longitude']].values

# data = cat_data.values + num_data
# data

ValueError: operands could not be broadcast together with shapes (103992,3) (103992,2) 

In [14]:
kmeans = KMeans(n_clusters=3)

groups = kmeans.fit_predict(cat_data)

In [15]:
df['pred_groups'] = groups

df.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,seas_reg_rare,pred_groups
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon,0
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0


In [17]:
df.pred_groups.value_counts(normalize=True)

0    0.539378
2    0.315149
1    0.145473
Name: pred_groups, dtype: float64

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
encoder = OneHotEncoder()

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

ValueError: Found unknown categories ['Antillean Nighthawk', 'Kalij Pheasant', 'Great Skua', 'Corn Crake', 'Little Curlew', 'Kauai Elepaio', 'Masked Duck', 'Passenger Pigeon', "Bulwer's Petrel"] in column 0 during transform

### Hashing Encoder

- breaking away from this cuz its not working

In [7]:
encoder = ce.HashingEncoder()

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [8]:
X_train

,name,season,RegionName,latitude,longitude
68458,Red-necked Phalarope,Fall,Heartland,45.177800,-93.501600
51708,House Finch,Spring,Mountain West,36.030221,-112.851305
43560,House Finch,Spring,Heartland,43.045578,-89.466100
22842,Mourning Dove,Winter,West Coast,32.759226,-117.221841
59836,Common Raven,Summer,Northwest,39.227499,-123.764377
...,...,...,...,...,...
103260,Northern Rough-winged Swallow,Summer,Appohzarka,34.724472,-92.761774
62751,Turkey Vulture,Spring,Southern Florida,26.949311,-81.030693
3344,Mallard,Spring,Mid-Atlantic North,39.397850,-76.986394
86682,Red-eyed Vireo,Spring,Northeast,42.368437,-74.439239


In [14]:
forest = RandomForestClassifier()

forest.fit(X_train, y_train)
forest.score(X_test, y_test)

ValueError: could not convert string to float: 'Northern Harrier'